In [1]:
!pip -q install llama-index==0.10.37

In [2]:
!pip -q install llama-index-llms-openai llama-index-llms-groq llama-index-experimental gradio fpdf

  Preparing metadata (setup.py) ... done


In [3]:
import pandas as pd
from pydantic import BaseModel, Field, field_validator
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
from google.colab import userdata
import textwrap

# ===== CONFIGURAÇÃO COM PYDANTIC V2 =====
class LLMConfig(BaseModel):
    model: str = Field(..., description="Nome do modelo Groq a ser usado")
    api_key: str = Field(..., description="Chave da API Groq")
    data_url: str = Field(..., description="URL do CSV com os dados")

    @field_validator("data_url")
    @classmethod
    def validar_url(cls, v: str) -> str:
        if not (v.startswith("http://") or v.startswith("https://")):
            raise ValueError("data_url deve começar com http:// ou https://")
        return v

    @field_validator("api_key")
    @classmethod
    def validar_api_key(cls, v: str) -> str:
        if not v or len(v.strip()) == 0:
            raise ValueError("api_key não pode ser vazia")
        return v

    model_config = {
        "extra": "allow",
        "json_schema_extra": {
            "example": {
                "model": "meta-llama/llama-4-scout-17b-16e-instruct",
                "api_key": "<SUA_CHAVE_AQUI>",
                "data_url": "https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
            }
        }
    }

# ===== Função para formatar texto =====
def formatar_texto(response, largura: int = 100, imprimir: bool = True):
    texto = response.response
    texto_formatado = textwrap.fill(texto, width=largura)
    if imprimir:
        print(texto_formatado)
    return texto_formatado

# ===== INICIALIZAÇÃO =====
# Obter a chave via Colab userdata
key = userdata.get('Groq_API')

# Configurar
config = LLMConfig(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key=key,
    data_url="https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
)

# Carregar CSV
df = pd.read_csv(config.data_url)

# Inicializar LLM via Settings
Settings.llm = Groq(model=config.model, api_key=config.api_key)

# Criar engine de consulta (somente aqui, depois do df estar pronto)
query_engine = PandasQueryEngine(df=df, verbose=True, synthesize_response=True)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.12/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


#Modularidade

Antes de criar o pipeline em si, você precisa definir alguns módulos.
No primeiro, temos algumas instruções do que é para ser feito com os dados. Depois, temos o Pandas Prompt, com informações sobre o DataFrame que estamos trabalhando. Por fim, temos um prompt de respostas, onde vai sintetizar a resposta que será gerada no final.

Mas, vamos entender o que precisamos fazer. Se estamos trabalhando com dados em português e queremos que o modelo só GAranta um bom resultado em português, a primeira coisa que precisamos fazer é alterar os textos todos para português. Dessa forma não haverá confusão, o modelo não se confundirá e responderá em inglês, como aconteceu na aula anterior.

In [4]:
from llama_index.core import PromptTemplate
from llama_index.experimental.query_engine.pandas import PandasInstructionParser

In [5]:
# Função para obter uma descrição das colunas do DataFrame
def descricao_colunas(df):
  descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
  return 'Aqui estão os detalhes das colunas do DataFrame:\n' + descricao


# Instruções para orientar o modelo a converter uma consulta em linguagem natural em código Python executável com a biblioteca Pandas
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO.\n"
    "5. Não coloque a expressão entre aspas.\n")

# Prompt que será enviado ao modelo para que ela gere o código Pandas desejado
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

# Prompt para guiar o modelo a sintetizar uma resposta com base nos resultados obtidos pela consulta Pandas
response_synthesis_prompt_str = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta:"
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

# Módulo para obter as instruções Pandas
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, colunas_detalhes=descricao_colunas(df), df_str=df.head(5)
)
# Módulo para executar as instruções Pandas
pandas_output_parser = PandasInstructionParser(df)

# Módulo para sintetizar a resposta
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

# Modelo
#llm = Groq(model='llama3-70b-8192', api_key=key)
#Carreguei um mais avançado

#Função eval() e instruções do modelo

`eval()` é uma função Python que serve basicamente para executar um código que esteja dentro de uma string. Ou seja, se temos um código dentro de um texto, vai pegá-lo e executar diretamente.

O código deve representar uma solução para a consulta. Então, se a pessoa fez uma pergunta, esse código vai ter que encontrar uma solução usando um código da biblioteca Pandas. Após, é definido que seja impresso apenas a expressão, que deve ser uma coisa importante de ser feita. Ele também pede para não colocar a expressão entre aspas no final, pois isso pode atrapalhar a execução da função eval().

Depois temos o pandas_prompt_str. Este é o prompt que será enviado para o modelo para que gere o código Pandas desejado. Nele, informamos ao modelo que está trabalhando com o dataframe do Pandas em Python chamado df, esse é o resultado do print df.head(). Repare que começa mostrando quais são as cinco primeiras linhas do arquivo para o modelo já ter um contexto. A partir disso, conseguimos fazer perguntas com maior facilidade.

Após, traz o df_str, que é uma variável que está abaixo, que é basicamente o df.head() para pegar as cinco primeiras linhas. Nisso, indica seguir as instruções. Em seguida, temos a Consulta {query str}, que é a consulta que faremos para o pipeline. E temos a expressão sendo gerada.

Por fim, temos esse response_synthesis_prompt_str, que serve para guiar o modelo a sintetizar uma resposta com base no resultado que foi obtido pela nossa consulta Pandas. Então, ele indica que "Dada uma pergunta de entrada, elabore uma resposta a partir dos resultados da consulta".

Após, traz qual foi a consulta feita, que é query_str. Também terá as instruções do Pandas que serão usadas para gerar a resposta, que é o pandas_instructions. Depois, ele tem essas instruções do Pandas, que são as instruções que serão usadas para chegar no resultado. Temos também a saída do Pandas, que é o pandas_output e a resposta.

---

# Para saber mais: modificando os prompts do pipeline

A capacidade de **customizar como as consultas são processadas e as respostas apresentadas** é essencial para atender às necessidades específicas dos usuários, aumentando a eficiência e a relevância dos resultados.

---

## Estrutura do pipeline

A primeira etapa antes de criar o pipeline é definir **3 módulos principais**:

1. **pandas\_prompt**

   * Traduz consultas em linguagem natural para comandos específicos do Pandas.
   * Facilita a manipulação direta dos dados.

2. **pandas\_output\_parser**

   * Executa os comandos gerados.
   * Aplica-os diretamente ao DataFrame para processar as informações.

3. **response\_synthesis\_prompt**

   * Elabora uma resposta clara e informativa com base nos resultados obtidos.
   * Essa resposta é a que será apresentada ao usuário.

---

## Código base

```python
# Módulo pandas_prompt
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str,
    df_str=df.head(5),
    colunas_detalhes=descricao_colunas(df)
)

# Módulo pandas_output_parser
pandas_output_parser = PandasInstructionParser(df)

# Módulo response_synthesis_prompt
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

# LLM
llm = Groq(model="llama3-70b-8192", api_key=key)
```

---

## Componentes essenciais

### 1. Instruction String

É o conjunto de instruções que orienta o modelo a converter consultas em linguagem natural para **código Python executável** utilizando Pandas.

🔑 Requisito importante: o código final deve ser uma expressão Python que possa ser chamada com `eval()`.

```python
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO.\n"
    "5. Não coloque a expressão entre aspas.\n"
)
```

---

### 2. Pandas Prompt String

Esse prompt é enviado ao modelo para gerar o código Pandas desejado.

Inclui:

* As **instruções detalhadas**.
* Uma **visão geral do DataFrame** (`df.head(5)`).
* **Detalhes das colunas** para dar mais contexto ao modelo.

Função auxiliar para descrever colunas:

```python
def descricao_colunas(df):
    descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return 'Aqui estão os detalhes das colunas do DataFrame:\n' + descricao
```

Prompt traduzido e ajustado:

```python
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)
```

---

### 3. Response Synthesis Prompt String

Responsável por guiar o modelo a sintetizar uma resposta explicativa, de forma natural, **atuando como um analista de dados**.

Inclui:

* Pergunta feita.
* Código Pandas utilizado.
* Saída obtida.
* Resposta em texto natural.

```python
response_synthesis_prompt_str = (
    "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
    "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
    "Consulta: {query_str}\n\n"
    "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
    "Saída do Pandas: {pandas_output}\n\n"
    "Resposta: \n\n"
    "Ao final, exibir o código usado em para gerar a resposta, no formato: O código utilizado foi `{pandas_instructions}`"
)
```

---

✅ Agora que os módulos estão definidos, o próximo passo é **construir o pipeline completo**.

---

##Construindo o pipeline de consulta

###Estruturando o pipeline

In [12]:
import pandas as pd
from pydantic import BaseModel, Field, field_validator
from llama_index.core import Settings, PromptTemplate
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.query_pipeline import QueryPipeline as QP, Link, InputComponent
from llama_index.experimental.query_engine.pandas import PandasInstructionParser
from google.colab import userdata
import textwrap

# ===== CONFIGURAÇÃO COM PYDANTIC V2 =====
class LLMConfig(BaseModel):
    model: str = Field(..., description="Nome do modelo Groq a ser usado")
    api_key: str = Field(..., description="Chave da API Groq")
    data_url: str = Field(..., description="URL do CSV com os dados")

    @field_validator("data_url")
    @classmethod
    def validar_url(cls, v: str) -> str:
        if not (v.startswith("http://") or v.startswith("https://")):
            raise ValueError("data_url deve começar com http:// ou https://")
        return v

    @field_validator("api_key")
    @classmethod
    def validar_api_key(cls, v: str) -> str:
        if not v or len(v.strip()) == 0:
            raise ValueError("api_key não pode ser vazia")
        return v

    model_config = {
        "extra": "allow",
        "json_schema_extra": {
            "example": {
                "model": "meta-llama/llama-4-scout-17b-16e-instruct",
                "api_key": "<SUA_CHAVE_AQUI>",
                "data_url": "https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
            }
        }
    }

# ===== Função para formatar texto =====
def formatar_texto(response, largura: int = 100, imprimir: bool = True):
    texto = response.response if hasattr(response, "response") else str(response)
    texto_formatado = textwrap.fill(texto, width=largura)
    if imprimir:
        print(texto_formatado)
    return texto_formatado

# ===== INICIALIZAÇÃO =====
# Obter a chave via Colab userdata
key = userdata.get('Groq_API')

# Configurar
config = LLMConfig(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    api_key=key,
    data_url="https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
)

# Carregar CSV
df = pd.read_csv(config.data_url)

# Inicializar LLM via Settings
Settings.llm = Groq(model=config.model, api_key=config.api_key)

# ===== PROMPTS =====
# Instruções para orientar o modelo
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO.\n"
    "5. Não coloque a expressão entre aspas.\n"
)

# Função para descrição das colunas
def descricao_colunas(df):
    descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return 'Aqui estão os detalhes das colunas do DataFrame:\n' + descricao

# Prompt Pandas
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

# Prompt de Resposta Rápida
response_synthesis_prompt_str = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta:\n"
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

# Módulo para obter as instruções Pandas
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, colunas_detalhes=descricao_colunas(df), df_str=df.head(5)
)

# Módulo para executar as instruções Pandas
pandas_output_parser = PandasInstructionParser(df)

# Módulo para sintetizar a resposta
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)


# ===== DEFINIÇÃO DOS NODES (usando QueryPipeline) =====
# Os nodes são representados pelos próprios componentes no QueryPipeline

# Criar o QueryPipeline
qp = QP(verbose=True)

# Adicionar modulos
qp.add_modules({
    "input": InputComponent(),
    "pandas_prompt": pandas_prompt,
    "llm": Settings.llm,
    "pandas_output_parser": pandas_output_parser,
    "response_synthesis_prompt": response_synthesis_prompt,
    "response_synthesizer": Settings.llm, # Use the LLM to synthesize the response
})

# Adicionar Links
qp.add_links(
    [
        Link("input", "pandas_prompt"),
        Link("pandas_prompt", "llm"),
        Link("llm", "pandas_output_parser"),
        Link("pandas_output_parser", "response_synthesis_prompt", dest_key="pandas_output"),
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link("llm", "response_synthesis_prompt", dest_key="pandas_instructions"), # Link LLM output (the code)
        Link("response_synthesis_prompt", "response_synthesizer"),
    ]
)

##Realizando consultas com a pipeline

Criamos uma variável chamada response e atribuímos um valor a ela. O processo será um pouco diferente, pois, ao invés de utilizar o Pandas Query Engine, estamos trabalhando diretamente com o pipeline.

Para isso, usaremos a variável `qp`, que definimos anteriormente. A execução do pipeline será feita por meio de `qp.run()`, passando como parâmetro o valor de query_str, que já definimos em etapas anteriores. Esse parâmetro é o destino do nosso primeiro link, presente tanto no prompt de resposta quanto no prompt da biblioteca Pandas.

Vamos perguntar: "Qual é a média gasta por cada tipo de cliente?" Temos clientes que são membros da nossa rede de varejo e clientes que não são membros. Será que um gasta mais que o outro, em média?

In [16]:
# ===== EXECUÇÃO =====
response = qp.run(query_str="Qual é a média gasta por cada tipo de cliente?")

> Running module input with input: 
query_str: Qual é a média gasta por cada tipo de cliente?

> Running module pandas_prompt with input: 
query_str: Qual é a média gasta por cada tipo de cliente?

> Running module llm with input: 
messages: Você está trabalhando com um dataframe do pandas em Python chamado `df`.
Aqui estão os detalhes das colunas do DataFrame:
`ID_compra`: object
`filial`: object
`cidade`: object
`tipo_cliente`: object
`...

> Running module pandas_output_parser with input: 
input: assistant: df.groupby('tipo_cliente')['total'].mean()

> Running module response_synthesis_prompt with input: 
query_str: Qual é a média gasta por cada tipo de cliente?
pandas_instructions: assistant: df.groupby('tipo_cliente')['total'].mean()
pandas_output: tipo_cliente
Membro    327.791305
Normal    318.122856
Name: total, dtype: float64

> Running module response_synthesizer with input: 
messages: Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir d

##Formatando a resposta


Na próxima célula, formatamos a resposta usando o módulo textwrap para melhorar a apresentação. Iniciamos criando uma variável chamada texto que será igual a response.message.content para obter a resposta.

Para formatar o texto, criamos uma variável texto_formatado que será igual a textwrap.fill(texto, width=100), sendo o width a largura. Por fim, exibimos o texto_formatado utilizando o print():

In [17]:
texto = response.message.content
texto_formatado = textwrap.fill(texto, width=100)
print(texto_formatado)

A média de gastos por tipo de cliente varia ligeiramente entre as categorias. Os clientes "Membro"
apresentam uma média de gasto de aproximadamente R$ 327,79, enquanto os clientes "Normal" gastam em
média R$ 318,12. Isso sugere que os membros tendem a gastar um pouco mais em comparação com os
clientes normais.  O código utilizado foi df.groupby('tipo_cliente')['total'].mean()


O código utilizado foi df.groupby('tipo_cliente')['total'].mean(). Isso é útil para a equipe da Zoop, pois além da resposta, podemos verificar se está tudo certo.

Como estamos utilizando uma LLM, podemos solicitar que ela forneça um insight sobre as diferenças de preço. Podemos testar outra consulta: "Por que clientes tipo membro têm maior média de gasto?" utilizando a variável response que será igual a qp.run(query_str="") e passamos a pergunta:

In [18]:
response = qp.run(query_str="Por que clientes do tipo membro tem maior média de gasto?")

> Running module input with input: 
query_str: Por que clientes do tipo membro tem maior média de gasto?

> Running module pandas_prompt with input: 
query_str: Por que clientes do tipo membro tem maior média de gasto?

> Running module llm with input: 
messages: Você está trabalhando com um dataframe do pandas em Python chamado `df`.
Aqui estão os detalhes das colunas do DataFrame:
`ID_compra`: object
`filial`: object
`cidade`: object
`tipo_cliente`: object
`...

> Running module pandas_output_parser with input: 
input: assistant: df.groupby('tipo_cliente')['total'].mean().sort_values(ascending=False)

> Running module response_synthesis_prompt with input: 
query_str: Por que clientes do tipo membro tem maior média de gasto?
pandas_instructions: assistant: df.groupby('tipo_cliente')['total'].mean().sort_values(ascending=False)
pandas_output: tipo_cliente
Membro    327.791305
Normal    318.122856
Name: total, dtype: float64

> Running module response_synthesizer with input: 
messages: 

Podemos exibir a resposta como fizemos anteriormente com o seguinte código:

In [19]:
texto = response.message.content
texto_formatado = textwrap.fill(texto, width=100)
print(texto_formatado)

Clientes do tipo membro apresentam uma média de gasto maior em comparação com clientes do tipo
normal. De acordo com os dados, a média de gasto para clientes membros é de aproximadamente R$
327,79, enquanto para clientes normais essa média fica em torno de R$ 318,12.   Essa diferença pode
ser resultado de vários fatores, como programas de fidelidade que incentivam membros a realizar
compras mais frequentes ou de maior valor, ou características demográficas e comportamentais que
distinguem os clientes membros dos normais.   O código utilizado foi
`df.groupby('tipo_cliente')['total'].mean().sort_values(ascending=False)`


Exercício: criando uma função para o pipeline de consultas

Nesta aula, aprendemos a criar um pipeline de consultas, que será crucial para a construção de uma aplicação futura. Para solidificar o aprendizado e preparar para usos práticos, sua tarefa é encapsular o pipeline em uma função, facilitando a reutilização e integração na aplicação que vamos começar a construir na próxima aula.

Dicas

Nomeie a função como pipeline_consulta.
A função deve aceitar um DataFrame chamado df como entrada.
Inclua a função descricao_colunas no corpo do código para ajudar na descrição das colunas do DataFrame.
A função deve retornar a variável qp, que representa o pipeline completo.

In [20]:
#Resposta da instrutora, a minha está no pipeline que usei:

def descricao_colunas(df):
    descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Aqui estão os detalhes das colunas do dataframe:\n" + descricao

# Definição de módulos da pipeline
def pipeline_consulta(df):
    instruction_str = (
        "1. Converta a consulta para código Python executável usando Pandas.\n"
        "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
        "3. O código deve representar uma solução para a consulta.\n"
        "4. IMPRIMA APENAS A EXPRESSÃO.\n"
        "5. Não coloque a expressão entre aspas.\n")

    pandas_prompt_str = (
        "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
        "{colunas_detalhes}\n\n"
        "Este é o resultado de `print(df.head())`:\n"
        "{df_str}\n\n"
        "Siga estas instruções:\n"
        "{instruction_str}\n"
        "Consulta: {query_str}\n\n"
        "Expressão:"
)

    response_synthesis_prompt_str = (
       "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
       "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
       "Consulta: {query_str}\n\n"
       "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
       "Saída do Pandas: {pandas_output}\n\n"
       "Resposta: \n\n"
       "Ao final, exibir o código usado em para gerar a resposta, no formato: O código utilizado foi `{pandas_instructions}`"
    )

    pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str,
    df_str=df.head(5),
    colunas_detalhes=descricao_colunas(df)
)

    pandas_output_parser = PandasInstructionParser(df)
    response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

    # Criação do Query Pipeline
    qp = QP(
        modules={
            "input": InputComponent(),
            "pandas_prompt": pandas_prompt,
            "llm1": llm,
            "pandas_output_parser": pandas_output_parser,
            "response_synthesis_prompt": response_synthesis_prompt,
            "llm2": llm,
        },
        verbose=True,
    )
    qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
    qp.add_links(
        [
            Link("input", "response_synthesis_prompt", dest_key="query_str"),
            Link("llm1", "response_synthesis_prompt", dest_key="pandas_instructions"),
            Link("pandas_output_parser", "response_synthesis_prompt", dest_key="pandas_output"),
        ]
    )
    qp.add_link("response_synthesis_prompt", "llm2")
    return qp